In [ ]:
import json
import csv
import re

In [ ]:
# Charger le lexique polonais (un mot par ligne)
with open("lexique_polonais.txt", encoding="utf-8") as f:
    lexique_polonais = set(word.strip().lower() for word in f if word.strip())

# Charger les données JSON
with open("transcriptions_ordonnées.json", encoding="utf-8") as f:
    data = json.load(f)

In [2]:
# Créer le fichier CSV de sortie
with open("interlangue.csv", "w", newline="", encoding="utf-8") as csvfile:
    writer = csv.writer(csvfile)
    # Écrire l'en-tête du tableau
    writer.writerow(["pays", "apprenant", "mot (interlangue)", "transcription automatique", "transcription manuelle"])

    derniere_transcription_auto = None  # pour suivre la dernière transcription automatique utilisée

    for exemple in data:
        apprenant = exemple.get("numéro d'apprenant", "")
        pays = exemple.get("pays", "")

        # Vérifier si la transcription automatique est en langue étrangère
        transcription_non_polonaise = exemple.get("transcription_automatique_non_pl", "").strip()
        if transcription_non_polonaise:
            print(f"Transcription non polonaise ignorée : {pays} {apprenant}")
            continue

        # Transcription automatique en polonais
        transcription_automatique = exemple.get("transcription automatique", "").strip()

        # Construire la transcription manuelle complète en joignant tous les segments
        segments_manuels = []
        for i in range(1, 20):  # On suppose qu'il n'y a pas plus de 20 segments
            clef = f"transcription manuelle ({i})"
            phrase = exemple.get(clef)
            if phrase:
                segments_manuels.append(phrase.strip())
        transcription_manuelle_complete = " ".join(segments_manuels)

        # Extraire les mots et les lignes à écrire
        lignes_a_ecrire = []

        for i in range(1, 20):
            clef = f"transcription manuelle ({i})"
            phrase = exemple.get(clef)
            if not phrase:
                continue

            mots = re.findall(r"\b\w+\b", phrase.lower())
            for mot in mots:
                if mot not in lexique_polonais:
                    lignes_a_ecrire.append([pays, apprenant, mot])

        # Écrire les lignes avec gestion des répétitions
        premiere_ligne = True
        for ligne in lignes_a_ecrire:
            transcription_auto_col = transcription_automatique if premiere_ligne and transcription_automatique != derniere_transcription_auto else ""
            transcription_man_col = transcription_manuelle_complete if premiere_ligne else ""
            writer.writerow(ligne + [transcription_auto_col, transcription_man_col])
            premiere_ligne = False

        # Mettre à jour la dernière transcription utilisée
        derniere_transcription_auto = transcription_automatique


Transcription non polonaise ignorée : FR 1114
Transcription non polonaise ignorée : FR 1105
Transcription non polonaise ignorée : IT 5114
Transcription non polonaise ignorée : IT 5105
Transcription non polonaise ignorée : IT 5120
Transcription non polonaise ignorée : IT 5109
Transcription non polonaise ignorée : UK 3103
Transcription non polonaise ignorée : UK 3118
Transcription non polonaise ignorée : UK 3101
Transcription non polonaise ignorée : UK 3104
Transcription non polonaise ignorée : UK 3105
Transcription non polonaise ignorée : UK 3106
Transcription non polonaise ignorée : GE 4101
Transcription non polonaise ignorée : GE 4118
Transcription non polonaise ignorée : GE 4108


In [3]:
# Comptage total des mots dans toutes les transcriptions manuelles
total_mots_manuels = 0

for exemple in data:
    for i in range(1, 20):  # mêmes limites que précédemment
        clef = f"transcription manuelle ({i})"
        phrase = exemple.get(clef)
        if phrase:
            mots = re.findall(r"\b\w+\b", phrase.lower())
            total_mots_manuels += len(mots)

print(f"\nNombre total de mots dans toutes les transcriptions manuelles : {total_mots_manuels}")



Nombre total de mots dans toutes les transcriptions manuelles : 3872
